In [1]:
import torch
from torch_geometric.data import Data
from poker_embeddings.poker_utils.datasets import UCIrvineDataset
import pandas as pd
from sklearn.model_selection import train_test_split
import timeit

In [2]:
def option1(cards_id, x, y):
    edges = []
    for i in range(len(cards_id)):
        for j in range(i + 1, len(cards_id)):
            id_i, id_j = cards_id[i].item(), cards_id[j].item()
            rank_i, rank_j = id_i // 4, id_j // 4
            suit_i, suit_j = id_i % 4, id_j % 4

            if suit_i == suit_j or abs(rank_i - rank_j) <= 1 or abs(rank_i - rank_j) == 12:
                edges += [(i, j), (j, i)]

    if not edges:
        edge_index = torch.empty((2, 0), dtype=torch.long)
    else:
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

    data = Data(x=x, edge_index=edge_index, y=y)
    return data

In [3]:
def option2(cards_id, x, y):
    card_pairs = torch.combinations(torch.arange(len(cards_id)), r=2, with_replacement=False)
    id_i, id_j = cards_id[card_pairs[:, 0]], cards_id[card_pairs[:, 1]]
    rank_i, rank_j = id_i // 4, id_j // 4
    suit_i, suit_j = id_i % 4, id_j % 4

    # Vectorized edge condition
    same_suit = suit_i == suit_j
    consecutive_rank = (abs(rank_i - rank_j) <= 1) | (abs(rank_i - rank_j) == 12)
    valid_edges = same_suit | consecutive_rank

    edges = card_pairs[valid_edges].T
    if edges.numel() == 0:
        edge_index = torch.empty((2, 0), dtype=torch.long)
    else:
        edge_index = edges.contiguous()

    data = Data(x=x, edge_index=edge_index, y=y)
    return data

In [4]:
def option3(cards_id, x, y):
    ranks = cards_id // 4
    suits = cards_id % 4
    num_cards = cards_id.size(0)
    idx_i = cards_id.unsqueeze(1).expand(num_cards, num_cards)
    idx_j = cards_id.unsqueeze(0).expand(num_cards, num_cards)

    ranks_i = ranks.unsqueeze(1).expand(num_cards, num_cards)
    ranks_j = ranks.unsqueeze(0).expand(num_cards, num_cards)

    suits_i = suits.unsqueeze(1).expand(num_cards, num_cards)
    suits_j = suits.unsqueeze(0).expand(num_cards, num_cards)

    suit_match = suits_i == suits_j
    rank_close = (torch.abs(ranks_i - ranks_j) <= 1) | (torch.abs(ranks_i - ranks_j) == 12)

    not_self = idx_i != idx_j

    edge_mask = (suit_match | rank_close) & not_self

    edge_index = edge_mask.nonzero(as_tuple=False).t().contiguous()

    data = Data(x=x, edge_index=edge_index, y=y)
    return data

In [5]:
X = pd.read_csv("data/uc_irvine/X.csv")
y = pd.read_csv("data/uc_irvine/y.csv")
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.6, random_state=29, stratify=y['CLASS']
    )
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [6]:
train_dataset = UCIrvineDataset(X_train, y_train, add_random_cards=True, use_card_ids=True,
                           graph=True, normalize_x=True)

In [7]:
train_dataset[0].y

tensor(0)

In [42]:
card_id = torch.tensor([1, 2, 3, 4, 5, 6, 7])
x = torch.tensor(
    [[1],[2],[3],[4],[5],[6],[7]]
)

In [59]:
option2(card_id, x,torch.tensor(0))

Data(x=[7, 1], edge_index=[2, 21], y=0)

In [60]:
total_time = timeit.timeit(lambda: option2(card_id, x, torch.tensor(0)))
print(f"Option 2 average time: {total_time / 1000000:.6f} seconds")

Option 2 average time: 0.000094 seconds


In [61]:
option3(card_id, x,torch.tensor(0))

Data(x=[7, 1], edge_index=[2, 42], y=0)

In [62]:
total_time = timeit.timeit(lambda: option3(card_id, x, torch.tensor(0)))
print(f"Option 3 average time: {total_time / 1000000:.6f} seconds")

Option 3 average time: 0.000046 seconds


In [63]:
option1(card_id, x,torch.tensor(0))

Data(x=[7, 1], edge_index=[2, 42], y=0)

In [64]:
total_time = timeit.timeit(lambda: option1(card_id, x, torch.tensor(0)))
print(f"Option 1 average time: {total_time / 1000000:.6f} seconds")

Option 1 average time: 0.000065 seconds
